# EXP_001 — ADC-24 CSV Recording Analysis

Plotting session data exported from the ADC-24 dashboard (FastAPI backend).

**Reference:** Mishra et al., *Sci. Robot.* 2024 — expected spontaneous spikes ~135 µV mean, up to ~1868 µV.

In [16]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter, find_peaks

## 1. Load & inspect data

In [17]:
CSV_PATH = "faaraday.csv"

df = pd.read_csv(CSV_PATH)
print(f"Columns: {list(df.columns)}")
print(f"Samples: {len(df)}")
print(f"Duration: {df['timestamp_s'].iloc[-1]:.1f} s ({df['timestamp_s'].iloc[-1]/60:.1f} min)")
print(f"\nVoltage (µV) stats:")
print(df['voltage_uv'].describe())

Columns: ['timestamp_s', 'channel', 'raw_adc', 'voltage_uv']
Samples: 1726
Duration: 207.1 s (3.5 min)

Voltage (µV) stats:
count    1726.000000
mean        2.356191
std         0.398132
min         1.097000
25%         2.311000
50%         2.348000
75%         2.380000
max        18.174000
Name: voltage_uv, dtype: float64


## 2. Raw signal — full trace with range slider

In [18]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=df['timestamp_s'],
    y=df['voltage_uv'],
    mode='lines',
    name='Raw signal',
    line=dict(color='#00d4aa', width=1),
))

fig1.update_layout(
    title='EXP_001 — Raw ADC-24 Recording (µV)',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (µV)',
    template='plotly_dark',
    height=500,
    hovermode='x unified',
    xaxis=dict(rangeslider=dict(visible=True)),
)

fig1.show()

## 3. Filtered signal (Savitzky-Golay) + peak detection

Using the same filter parameters as Mishra et al.:
- Savitzky-Golay, 3rd order, window = 11
- Peak prominence ≥ 10 µV

In [19]:
voltage = df['voltage_uv'].values
time = df['timestamp_s'].values

# Savitzky-Golay filter (Mishra et al. parameters)
filtered = savgol_filter(voltage, window_length=11, polyorder=3)

# Baseline subtraction
baseline = np.mean(filtered)
centered = filtered - baseline

# Peak detection
PROMINENCE_THRESHOLD = 10  # µV
pos_peaks, pos_props = find_peaks(centered, prominence=PROMINENCE_THRESHOLD)
neg_peaks, neg_props = find_peaks(-centered, prominence=PROMINENCE_THRESHOLD)

print(f"Baseline: {baseline:.2f} µV")
print(f"Positive peaks found: {len(pos_peaks)}")
print(f"Negative peaks found: {len(neg_peaks)}")
if len(pos_peaks) > 0:
    print(f"  Max positive spike: {centered[pos_peaks].max():.1f} µV")
    print(f"  Mean positive spike: {centered[pos_peaks].mean():.1f} µV")
if len(neg_peaks) > 0:
    print(f"  Max negative spike: {centered[neg_peaks].min():.1f} µV")

duration_s = time[-1] - time[0]
total_peaks = len(pos_peaks) + len(neg_peaks)
freq = total_peaks / duration_s if duration_s > 0 else 0
print(f"\nSpiking frequency: {freq:.3f} Hz ({freq*60:.1f} spikes/min)")
print(f"  (Mishra et al. mean: 0.12 Hz = 7.2 spikes/min)")

Baseline: 2.36 µV
Positive peaks found: 0
Negative peaks found: 0

Spiking frequency: 0.000 Hz (0.0 spikes/min)
  (Mishra et al. mean: 0.12 Hz = 7.2 spikes/min)


In [20]:
fig2 = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=['Raw signal', 'Filtered + peaks (baseline-subtracted)'],
)

# Row 1: raw
fig2.add_trace(go.Scatter(
    x=time, y=voltage,
    mode='lines', name='Raw',
    line=dict(color='#868e96', width=0.8),
), row=1, col=1)

# Row 2: filtered + peaks
fig2.add_trace(go.Scatter(
    x=time, y=centered,
    mode='lines', name='Filtered (SG)',
    line=dict(color='#00d4aa', width=1.2),
), row=2, col=1)

# Positive peaks
if len(pos_peaks) > 0:
    fig2.add_trace(go.Scatter(
        x=time[pos_peaks], y=centered[pos_peaks],
        mode='markers', name=f'+ peaks ({len(pos_peaks)})',
        marker=dict(color='#ffd43b', size=8, symbol='triangle-up'),
    ), row=2, col=1)

# Negative peaks
if len(neg_peaks) > 0:
    fig2.add_trace(go.Scatter(
        x=time[neg_peaks], y=centered[neg_peaks],
        mode='markers', name=f'− peaks ({len(neg_peaks)})',
        marker=dict(color='#ff6b6b', size=8, symbol='triangle-down'),
    ), row=2, col=1)

# Reference lines
fig2.add_hline(y=135, line_dash='dash', line_color='#ffd43b',
               annotation_text='Mishra mean spike (135 µV)',
               annotation_position='top left', row=2, col=1)
fig2.add_hline(y=-135, line_dash='dash', line_color='#ffd43b', row=2, col=1)

fig2.update_layout(
    title='EXP_001 — Signal Processing (Mishra et al. parameters)',
    template='plotly_dark',
    height=700,
    hovermode='x unified',
    showlegend=True,
)
fig2.update_yaxes(title_text='Voltage (µV)', row=1, col=1)
fig2.update_yaxes(title_text='Voltage (µV)', row=2, col=1)
fig2.update_xaxes(title_text='Time (s)', row=2, col=1)

fig2.show()

## 4. Peak amplitude distribution

In [21]:
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Full signal distribution', 'Peak amplitudes'],
)

# Full signal histogram
fig3.add_trace(go.Histogram(
    x=centered, nbinsx=80,
    marker_color='#00d4aa', opacity=0.7,
    name='All samples',
), row=1, col=1)

# Peak amplitudes
all_peak_amplitudes = []
all_peak_colors = []
if len(pos_peaks) > 0:
    all_peak_amplitudes.extend(centered[pos_peaks].tolist())
    all_peak_colors.extend(['#ffd43b'] * len(pos_peaks))
if len(neg_peaks) > 0:
    all_peak_amplitudes.extend(centered[neg_peaks].tolist())
    all_peak_colors.extend(['#ff6b6b'] * len(neg_peaks))

if all_peak_amplitudes:
    fig3.add_trace(go.Histogram(
        x=all_peak_amplitudes, nbinsx=30,
        marker_color='#748ffc', opacity=0.8,
        name='Peak amplitudes',
    ), row=1, col=2)

fig3.update_layout(
    title='EXP_001 — Amplitude Distributions',
    template='plotly_dark',
    height=400,
)
fig3.update_xaxes(title_text='Voltage (µV)', row=1, col=1)
fig3.update_xaxes(title_text='Peak voltage (µV)', row=1, col=2)
fig3.update_yaxes(title_text='Count', row=1, col=1)
fig3.update_yaxes(title_text='Count', row=1, col=2)

fig3.show()

## 5. Spiking frequency over time

In [22]:
# Compute spiking frequency in sliding windows
WINDOW_S = 30  # 30-second windows (same as Mishra et al.)

all_peak_times = np.sort(np.concatenate([time[pos_peaks], time[neg_peaks]])) if (len(pos_peaks) + len(neg_peaks)) > 0 else np.array([])

if len(all_peak_times) > 0:
    t_start = time[0]
    t_end = time[-1]
    window_centers = []
    window_freqs = []
    
    t = t_start + WINDOW_S / 2
    while t < t_end - WINDOW_S / 2:
        n = np.sum((all_peak_times >= t - WINDOW_S/2) & (all_peak_times < t + WINDOW_S/2))
        window_centers.append(t)
        window_freqs.append(n / WINDOW_S)
        t += WINDOW_S / 2  # 50% overlap
    
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(
        x=window_centers, y=window_freqs,
        mode='lines+markers', name='Spiking freq',
        line=dict(color='#748ffc', width=2),
        marker=dict(size=6),
    ))
    fig4.add_hline(y=0.12, line_dash='dash', line_color='#ffd43b',
                   annotation_text='Mishra mean (0.12 Hz)')
    fig4.update_layout(
        title=f'EXP_001 — Spiking Frequency ({WINDOW_S}s sliding window)',
        xaxis_title='Time (s)',
        yaxis_title='Frequency (Hz)',
        template='plotly_dark',
        height=400,
    )
    fig4.show()
else:
    print('No peaks detected — cannot compute spiking frequency.')

No peaks detected — cannot compute spiking frequency.
